In [1]:
# Introduce imports which we need
import pandas as pd
import numpy as np

In [2]:
# Load data exported from Wharton
# Industry INDL
df_fundamentals_indl = pd.read_csv('/Users/sebastiansydow/8_FM/fundamentals_indl.csv')
# Industry FS
# df_fundamentals_fs = pd.read_csv('/Users/sebastiansydow/8_FM/02_data_fundamentals_industry_fs.csv')

In [4]:
df_fundamentals_indl[df_fundamentals_indl['cusip']=='03783310']

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,xsga,costat,ggroup,gind,gsector,gsubind,sic,spcindcd,spcseccd,ipodate


In [3]:
def compute_financial_ratios(df):
    
    ############################### Profitability #################################
    ## Gross Profit Margin: Gross Profit divided by Sales
    df['Gross_Profit_Margin'] = df['gp'] / df['sale']
    
    
    ## SG&A Expense Ratio: Sales, General & Administrative Expenses divided by Sales
    df['SG&A_Expense_Ratio'] = df['xsga'] / df['sale']
    
    
    ## R&D Expense Ratio: Research & Development Expenses divided by Sales
    df['R&D_Expense_Ratio'] = df['xrd'] / df['sale']
    
    
    ## Depreciation Expense Ratio: Depreciation Expenses divided by Sales
    df['Depreciation_Expense_Ratio'] = df['dp'] / df['sale']
    
    
    ## Interest Expense Ratio: Interest and Related Expenses divided by Sales
    df['Interest_Expense_Ratio'] = df['xint'] / df['sale']
    
    
    ## Net Profit Margin: Net Income divided by Sales
    df['Net_Profit_Margin'] = df['ni'] / df['sale']
    
    
    ## Return on Equity
    # Shift to allow for Calculation of Return on Equity
    df['prev_Total_Stockholders_Equity'] = df.groupby('cusip')['teq'].shift()

    # Calculation
    # Return on Equity: Net Income divided by Average of Total Equity current year and Total Equity Stock prior year 
    df['Return_on_Equity'] = df['ni'] / ( (df['teq'] +  df['prev_Total_Stockholders_Equity']) /2)
    
    # Drop Column 
    df = df.drop(['prev_Total_Stockholders_Equity'], axis=1)
    
    
    ## Return on Capital Employed
    # Shift to allow for Calculation
    df['prev_Total_Assets'] = df.groupby('cusip')['at'].shift()
    df['prev_Current_Liabilities'] = df.groupby('cusip')['lct'].shift()
    
    # Calculation
    # Return on Capital Employed = Net Income divided by the average of opening and closing Capital Employed
    df['Return_on_Capital_Employed'] = df['ni'] / ( ( (df['at'] - df['lct']) + 
                                                    (df['prev_Total_Assets'] - df['prev_Current_Liabilities']) ) /2)
    
    # Drop Column 
    df = df.drop(['prev_Total_Assets'], axis=1)
    df = df.drop(['prev_Current_Liabilities'], axis=1)
    
    
    ############################ Capital Structure #################################
    ## Debt to Equity Ratio: Debt divided by Equity
    df['D/E_Ratio'] = df['dt'] / df['teq']
    
    
    ## Long-Term Debt / Net Income: 
    df['Long_Term_Debt_to_Net_Income'] = df['dltt'] / df['ni']

    
    ## Fixed Assets to Total Assets: Property, Plant & Equipment divided by Total Assets
    df['Fixed_Assets_to_Total_Assets'] = df['ppent'] / df['at']
    
    
    
    ############################## Cash Flow #################################
    ## Free Cash Flow to Equity_1: Operating Cash Flow - Capital Expenditures + Net Debt Issued
    df['FCFE_1'] = df['oancf'] - df['capx'] + (df['dltis']-df['dltr'])
        
    
    ## Growth Rate of FCFE_1
    # Shift to allow for Calculation of Growth Rate
    df['prev_FCFE_1'] = df.groupby('cusip')['FCFE_1'].shift()
    
    # Calculation
    # Change in FCFE_1: FCFE_1 current year divided by FCFE_1 prior year minus 1 
    df['Rel_Change_FCFE_1'] = (df['FCFE_1'] / df['prev_FCFE_1']) - 1
    
    # Drop Column 
    df = df.drop(['prev_FCFE_1'], axis=1)
    
    
    ## Free Cash Flow to Equity_2: Operating Cash Flow + Investing Cash Flow + Net Debt Issued
    df['FCFE_2'] = df['oancf'] + df['ivncf'] + (df['dltis']-df['dltr'])

        
    ## Growth Rate of FCFE_2
    # Shift to allow for Calculation of Growth Rate
    df['prev_FCFE_2'] = df.groupby('cusip')['FCFE_2'].shift()
    
    # Calculation
    # Change in FCFE_2: FCFE_2 current year divided by FCFE_2 prior year minus 1 
    df['Rel_Change_FCFE_2'] = (df['FCFE_2'] / df['prev_FCFE_2']) - 1
    
    # Drop Column 
    df = df.drop(['prev_FCFE_2'], axis=1)
    
        
    ## Cash Flow Coverage Ratio: Operation Cash Flow / Total Debt
    df['Cash_Flow_Coverage_Ratio'] = df['oancf'] / df['dt']
    
    
    
    ############################ Payout Structure ############################
    ## Dividend Payout Ratio: Dividents Paid divided by Net Income
    df['Dividend_Payout_Ratio'] = df['dv'] / df['ni']
    

    ## Growth Rate of Retained Earnings
    # Shift to allow for Calculation of Growth Rate
    df['prev_Retained_Earnings'] = df.groupby('cusip')['re'].shift()
    
    # Calculation
    # Change in Retained Earnings: Retained Earnings current year divided by Retained Earnings prior year minus 1
    df['Rel_Change_Retained_Earnings'] = (df['re'] / df['prev_Retained_Earnings']) - 1
    
    # Drop Column 
    df = df.drop(['prev_Retained_Earnings'], axis=1)
    

    ## Growth Rate of Treasury Stock
    # Shift to allow for Calculation of Growth Rate
    df['prev_Treasury_Stock'] = df.groupby('cusip')['tstk'].shift()
    
    # Calculation
    # Change in Treasury Stock: Treasury Stock current year divided by Treasury Stock prior year minus 1
    df['Rel_Change_Treasury_Stock'] = (df['tstk'] / df['prev_Treasury_Stock']) - 1
    
    # Drop Column 
    df = df.drop(['prev_Treasury_Stock'], axis=1)
    
    
    ## Growth Rate of Earnings per Share
    df['prev_Earnings_per_Share'] = df.groupby('cusip')['epspx'].shift()
    
    # Calculation
    # Change in Earnings per Share: Treasury Stock current year divided by Treasury Stock prior year minus 1
    df['Rel_Change_Earnings_per_Share'] = (df['epspx'] / df['prev_Earnings_per_Share']) - 1
    
    # Drop Column 
    df = df.drop(['prev_Earnings_per_Share'], axis=1)
    
    return df

In [4]:
# Calculate Financial Ratios
df_fundamentals_indl_calc = compute_financial_ratios(df_fundamentals_indl)

# df_fundamentals_fs_calc = compute_financial_ratios(df_fundamentals_fs)

In [5]:
def add_prev_years(df, n, list_metrics):
    
    """ xxx

    Parameters
    ----------
    xxxx
    Returns
    -------
    xxx
    """
    
    # Sanity Check
    if type(list_metrics) != list:
        print('Metrics must be of type "list"!')
        return None 
    
    if type(n) != int:
        print('n must be of type "int"!')
        return None
    
    # Calculate a list of years for which the financial ratios need to be added as a separate column
    list_years = list(range(0, n + 1))
    
    # Add columns for each metric to the dataframe
    for metric in list_metrics:
        for year in list_years:
            df[metric + str('_t') + str(year)] = df.groupby('cusip')[metric].shift(year)
            if year == list_years[-1]:
                df[metric + str('_avg')] = df.iloc[:, (-n-1):].mean(axis=1) # calculate & add mean
                df[metric + str('_std')] = df.iloc[:, (-n-2):-1].std(axis=1) # calculate & add standard deviation
                df = df.drop([metric], axis=1) # drop metric column as it is now already included with metric_t0 
            
    return df

In [6]:
# Find index of first financial ratio, which we calculated
index_metrics = df_fundamentals_indl_calc.columns.get_loc('Gross_Profit_Margin')

# Retrieve column names of all financial ratios
metrics = df_fundamentals_indl_calc.columns[index_metrics:]

In [7]:
df_fundamentals_indl_final = add_prev_years(df_fundamentals_indl_calc, 9, list(metrics))

# df_fundamentals_fs_final = compute_financial_ratios(df_fundamentals_fs_calc, 9, metrics)

In [10]:
df_fundamentals_indl_final

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,Rel_Change_Earnings_per_Share_t2,Rel_Change_Earnings_per_Share_t3,Rel_Change_Earnings_per_Share_t4,Rel_Change_Earnings_per_Share_t5,Rel_Change_Earnings_per_Share_t6,Rel_Change_Earnings_per_Share_t7,Rel_Change_Earnings_per_Share_t8,Rel_Change_Earnings_per_Share_t9,Rel_Change_Earnings_per_Share_avg,Rel_Change_Earnings_per_Share_std
0,1000,1964/12/31,1964,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,1965/12/31,1965,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.750000,NaN
2,1000,1966/12/31,1966,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.864474,2.666537
3,1000,1967/12/31,1967,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,-5.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.056603,2.348016
4,1000,1968/12/31,1968,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,-1.978947,-5.75,NaN,NaN,NaN,NaN,NaN,NaN,-3.188793,1.935290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428183,339965,2021/01/31,2020,INDL,C,D,STD,SNOW,833445109,SNOWFLAKE INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.754216,0.303397
428184,345920,2018/12/31,2018,INDL,C,D,STD,HYFM,44888K209,HYDROFARM HLDNG GP INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428185,345920,2019/12/31,2019,INDL,C,D,STD,HYFM,44888K209,HYDROFARM HLDNG GP INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.211538,NaN
428186,345980,2018/12/31,2018,INDL,C,D,STD,WISH,21077C107,CONTEXTLOGIC INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Save files
df_fundamentals_indl_final.to_csv('/Users/sebastiansydow/8_FM/01_b_data_fundamentals_indl_final_calculated.csv') 
# df_fundamentals_fs_final.to_csv('/Users/sebastiansydow/8_FM/02_b_data_fundamentals_fs_calculated') 